<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Werner González
- Nombre de alumno 2: Maximiliano Rosadio


### **Link de repositorio de GitHub:** `https://github.com/wernergh9/MDS7202.git`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [38]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

!pip install xgboost
from xgboost import XGBClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')

#extras
!pip install mlxtend
from sklearn.pipeline import FeatureUnion
from mlxtend.preprocessing import DenseTransformer

Requirement already up-to-date: plotly in c:\users\werne\anaconda3\lib\site-packages (5.11.0)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\werne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\werne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [5]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia
df_comics = df_comics.drop("Unnamed: 0", axis=1) # eliminar primera columna innecesaria (?)

In [6]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,55,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1362,Zatanna,Zatanna Zatara,Zatanna Zatara,10,Zatanna is the daughter of adventurer John Zat...,Zatanna is genetically talented with her magi...,90,10,25,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1363,Zero,DWN-∞: Zero,DWN-∞: Zero,18,Zero was created by the late Dr. Albert Wily ...,NaN,80,100,100,100,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1364,Zoom (New 52),Hunter Zolomon,NaN,20,"Hunter Zolomon is better known as Zoom, a spee...",After tricking Barry Allen and Wally West into...,95,50,100,75,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1365,Zoom,Hunter Zolomon,Hunter Zolomon,9,Hunter Zolomon had a troubled relationship wi...,"Zoom is able to alter time, to make himself ev...",75,10,100,30,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [7]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [8]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\werne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [9]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [10]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [11]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [12]:
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score'
                        , 'durability_score', 'power_score', 'combat_score']

In [13]:
# Define custom transformer
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):

        self.columns = columns

        if len(columns) == 1:
            self.columns = columns[0]

    def fit(self, X, y=None):
        return self

    def transform(self, X):

        return X[self.columns]

# Define categorical pipeline
bow_pipe = Pipeline([
    ('selector', ColumnSelector(["history_text"])),
    ('bag_of_words', CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))),
    ("toDense", DenseTransformer())
])

# Define numerical pipeline
num_pipe = Pipeline([
    ('selector', ColumnSelector(atributos_de_interes)),
    ('scaler', MinMaxScaler())
])

# Fit feature union to training data
preprocessor = FeatureUnion([
    ('cat', bow_pipe),
    ('num', num_pipe)
])

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [14]:
#### Código aquí ####
pipe = Pipeline([
    ('Preprocesamiento', preprocessor),
    ('SelectPercentile', SelectPercentile(percentile=90)),
    ('Classifier', MultinomialNB())
])

X_train, X_test, y_train, y_test = train_test_split(
    df_comics.drop(columns=["alignment"]), df_comics['alignment'], test_size=0.25, shuffle=True)

pipe = pipe.fit(X_train, y_train)

In [15]:
y_pred = pipe.predict(X_test)

In [16]:
print("Classification Report\n\n",classification_report(y_test, y_pred))
print("")
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred), "\n")

Classification Report

               precision    recall  f1-score   support

         Bad       0.83      0.09      0.16       115
        Good       0.56      0.99      0.72       177
     Neutral       0.00      0.00      0.00        30

    accuracy                           0.57       322
   macro avg       0.47      0.36      0.29       322
weighted avg       0.61      0.57      0.45       322


Matriz de confusión

 [[ 10 105   0]
 [  2 175   0]
 [  0  30   0]] 



C:\Users\werne\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\werne\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\werne\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
#### Código aquí ####
pipe2 = Pipeline([
    ('Preprocesamiento', preprocessor),
    ('SelectPercentile', SelectPercentile(percentile=90)),
    ('Classifier', DummyClassifier(strategy="stratified"))
])

pipe2 = pipe2.fit(X_train, y_train)

In [18]:
y_pred2 = pipe2.predict(X_test)

print("Classification Report\n\n",classification_report(y_test, y_pred2))
print("")
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred2), "\n")

Classification Report

               precision    recall  f1-score   support

         Bad       0.39      0.31      0.35       115
        Good       0.57      0.63      0.60       177
     Neutral       0.11      0.13      0.12        30

    accuracy                           0.47       322
   macro avg       0.36      0.36      0.36       322
weighted avg       0.47      0.47      0.47       322


Matriz de confusión

 [[ 36  64  15]
 [ 49 112  16]
 [  7  19   4]] 



```
El clasificador MltinomialNB es uno de las clasificadores más antiguos usados en NLP y se podría considerar bastante básico, ya que para clasificar lo único que hace es realizar un conteo de los n-gramas de cada clase y ver cual tiene mayor porbabilidad. El problema de esto, es que las porbabilidades pueden ser muy bajas cuando la dimensión de los datos crece mucho, de modo que podría ser mejor utilizar un modelo más complejo como ensamblaje de modelos y en una búsqueda en grilla para ver cuál es el mejor modelo con cuáles hiperparámetros. Otra forma para poder mejorar el resultado de la clasificación podría ser usar un mejor modelo para presentar los documentos (como embbedings o TDIF en vez de BOW). 

Con respecto al desempeño del modelo, vemos que alcanza un accuracy mayor al del modelo dummy lo cual es lo mínimo a esperar en un modelo de clasificación. Sin embargo, dado el desbalanceo de clases, el modelo no aprendió a clasificar a la clase neutro.

Se puede concluir de esta parte dos cosas importantes:

* Se pueden obtener mejores resultados con un modelo que adivinando
* Estos resultados se podrían mejorar aún más usando distintas técnicas y probando distintos modelos y sus configuraciones
```

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_comics.drop(columns=["alignment"]), df_comics['alignment'].replace(['Bad', 'Good', 'Neutral'],[0, 1, 2]), 
    test_size=0.25, shuffle=True)

In [57]:
#### Código aquí ####

clf = Pipeline([
    ('Preprocesamiento', preprocessor),
    ('SelectPercentile', SelectPercentile(percentile=90)),
    ('classifier', MultinomialNB())
])

param_grid = [
    {
        "Preprocesamiento__cat__bag_of_words__ngram_range": [(1,1), (1,2), (1,3)],
        "SelectPercentile__percentile": [20, 40, 60, 80],
        "classifier__alpha": [0.1, 1.0, 10.0],
        "classifier": [MultinomialNB()]
    },
    {
        "Preprocesamiento__cat__bag_of_words__ngram_range": [(1,1), (1,2), (1,3)],
        "SelectPercentile__percentile": [20, 40, 60, 80],
        "classifier__n_estimators": [2,3,5],
        "classifier__gamma": [0.5, 1, 1.5, 2, 5],
        "classifier__max_depth": [3, 4, 5],
        "classifier": [XGBClassifier()]
    },
    {
        "Preprocesamiento__cat__bag_of_words__ngram_range": [(1,1), (1,2), (1,3)],
        "SelectPercentile__percentile": [20, 40, 60, 80],
        "classifier__n_estimators": [200, 500],
        "classifier__max_depth": [4,5,6,7,8],
        "classifier__criterion": ['gini', 'entropy'],
        "classifier": [RandomForestClassifier()]
    }
]

grid_search = HalvingGridSearchCV(clf, param_grid, cv=3, verbose=10,n_jobs=-1)
grid_search.fit(X_train, y_train)

n_iterations: 4
n_required_iterations: 7
n_possible_iterations: 4
min_resources_: 18
max_resources_: 963
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 816
n_resources: 18
Fitting 3 folds for each of 816 candidates, totalling 2448 fits
----------
iter: 1
n_candidates: 272
n_resources: 54
Fitting 3 folds for each of 272 candidates, totalling 816 fits
----------
iter: 2
n_candidates: 91
n_resources: 162
Fitting 3 folds for each of 91 candidates, totalling 273 fits
----------
iter: 3
n_candidates: 31
n_resources: 486
Fitting 3 folds for each of 31 candidates, totalling 93 fits


HalvingGridSearchCV(cv=3,
                    estimator=Pipeline(steps=[('Preprocesamiento',
                                               FeatureUnion(transformer_list=[('cat',
                                                                               Pipeline(steps=[('selector',
                                                                                                ColumnSelector(columns='history_text')),
                                                                                               ('bag_of_words',
                                                                                                CountVectorizer(ngram_range=(1,
                                                                                                                             2),
                                                                                                                tokenizer=<__main__.StemmerTokenizer object at 0x0000023086432EE0>)),
                                                                                               ('toDense',
                                                                                                DenseTransformer())])),
                                                                              ('num',
                                                                               Pipeline(steps=[...
                                 'classifier__max_depth': [3, 4, 5],
                                 'classifier__n_estimators': [2, 3, 5]},
                                {'Preprocesamiento__cat__bag_of_words__ngram_range': [(1,
                                                                                       1),
                                                                                      (1,
                                                                                       2),
                                                                                      (1,
                                                                                       3)],
                                 'SelectPercentile__percentile': [20, 40, 60,
                                                                  80],
                                 'classifier': [RandomForestClassifier()],
                                 'classifier__criterion': ['gini', 'entropy'],
                                 'classifier__max_depth': [4, 5, 6, 7, 8],
                                 'classifier__n_estimators': [200, 500]}],
                    verbose=10)

In [58]:
grid_search.best_params_ 

{'Preprocesamiento__cat__bag_of_words__ngram_range': (1, 1),
 'SelectPercentile__percentile': 60,
 'classifier': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=2, gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=5, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               n_estimators=5, n_jobs=None, num_parallel_tree=None,
               predictor=None, random_state=None, ...),
 'classifier__gamma': 2,
 'classifier__max_depth': 5,
 'classifier__n_estimators': 5}

In [79]:
y_pred_gs = grid_search.predict(X_test)

print("Classification Report\n\n",classification_report(y_test, y_pred_gs))
print("")
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred_gs), "\n")

Classification Report

               precision    recall  f1-score   support

           0       0.66      0.35      0.46       110
           1       0.63      0.93      0.75       178
           2       0.00      0.00      0.00        34

    accuracy                           0.64       322
   macro avg       0.43      0.43      0.41       322
weighted avg       0.58      0.64      0.57       322


Matriz de confusión

 [[ 39  71   0]
 [ 11 166   1]
 [  9  25   0]] 



```
Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

La mejor combinación encontrada fue para el clasificador XGBoost, con gamma=0.2, max_depth=5 y n_estimators=5. 

Por otro lado resultó que la cantidad de columnas seleccionadas por SelectPercentile es el 60% de las características con mayor score y los parámetros de CountVectorizer fue de (1,1), lo que significa que usar solo unigramas obtuvo el mayor score de todas las combinaciones posibles. Esto se traduce a que la palabras del texto por si solas fueron suficiente para crear el mejor clasificador por sobre bi-gramas y tri-gramas.
```



## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [71]:
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text']) # eliminar ejemplos sin historia
df_comics_no_label = df_comics_no_label.drop("Unnamed: 0", axis=1) # eliminar primera columna innecesaria (?)
df_comics_no_label = df_comics_no_label.reset_index(drop=True)

In [76]:
label_dict = {0: 'Bad',
              1: 'Good',
              2: 'Neutral'}

In [80]:
y_pred_gs = grid_search.predict(df_comics_no_label.drop(columns=["alignment"]))
print("      Vergil: ",label_dict[y_pred_gs[list(df_comics_no_label["name"]).index("Vergil")]])
print("Gorilla Girl: ",label_dict[y_pred_gs[list(df_comics_no_label["name"]).index("Gorilla Girl")]])
print("      Batcow: ",label_dict[y_pred_gs[list(df_comics_no_label["name"]).index("Batcow")]])

      Vergil:  Good
Gorilla Girl:  Good
      Batcow:  Good


<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>